In [ ]:
# import torch
# from models.pinal_module.T2stru_loading import load_T2Struc

# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

# T2Struc, text_tokenizer, structure_tokenizer = load_T2Struc("/t9k/mnt/CM/text2amp/weights/T2struc-fined/2025-11-27_21-54-10/final_model", dtype=torch.float16, device=DEVICE)
# #
# print(T2Struc)

In [ ]:
# from models.progen3_module.modeling import ProGen3ForCausalLM

# progen = ProGen3ForCausalLM.from_pretrained("./weights/progen3-762m", torch_dtype=torch.float16, device_map="cuda")

# print(progen)

In [ ]:
# import os
# import shutil
# import torch

# from transformers import T5EncoderModel
# from models.pinal_module.T2stru_loading import load_T2Struc


# def export_lm_only(
#     t2struc_final_model_dir: str,
#     pinal_official_t5_large_ckpt_dir: str,
#     lm_out_dir: str,
#     device: torch.device,
#     dtype: torch.dtype = torch.float16,
# ):
#     """
#     导出 T2Struc.lm (T5EncoderModel) 为独立 HuggingFace 格式目录：
#       lm_out_dir/
#         - config.json
#         - model.safetensors 或 pytorch_model.bin
#         - tokenizer相关文件（tokenizer.json / spiece.model / special_tokens_map.json / tokenizer_config.json 等）

#     注意：
#     - pinal_official_t5_large_ckpt_dir：用于在 tokenizer 无法 save_pretrained 时回退复制相关文件，
#       其语义是“PINAL authors 官方训练并发布的 T5-large 子模块/权重目录”。
#     """
#     os.makedirs(lm_out_dir, exist_ok=True)

#     # 1) 按现有方式加载完整模型（确保 lm 权重已被正确加载）
#     T2Struc, text_tokenizer, structure_tokenizer = load_T2Struc(
#         t2struc_final_model_dir,
#         dtype=dtype,
#         device=device,
#     )
#     lm: T5EncoderModel = T2Struc.lm
#     lm.eval()

#     # 2) 保存 lm 权重 + config（HF 标准）
#     #    safe_serialization=True 会优先产出 model.safetensors（环境没装 safetensors 时会退回 pytorch_model.bin）
#     lm.save_pretrained(lm_out_dir, safe_serialization=True)

#     # 3) 保存 tokenizer（优先用 text_tokenizer.save_pretrained；不行就复制 pinal_official_t5_large_ckpt_dir 里的 tokenizer 文件）
#     saved_tok = False
#     if hasattr(text_tokenizer, "save_pretrained"):
#         try:
#             text_tokenizer.save_pretrained(lm_out_dir)
#             saved_tok = True
#         except Exception as e:
#             print(
#                 "[WARN] text_tokenizer.save_pretrained 失败，将回退复制 pinal_official_t5_large_ckpt_dir。"
#                 f"原因: {e}"
#             )

#     if not saved_tok:
#         # 把 pinal-official-t5-large 目录里的 tokenizer & config 相关文件复制过来
#         # 注意：lm.save_pretrained 已经写了 config.json；这里复制时允许覆盖/合并
#         for fn in [
#             "config.json",
#             "special_tokens_map.json",
#             "spiece.model",
#             "tokenizer_config.json",
#             "tokenizer.json",
#         ]:
#             src = os.path.join(pinal_official_t5_large_ckpt_dir, fn)
#             if os.path.exists(src):
#                 shutil.copy2(src, os.path.join(lm_out_dir, fn))

#     print(f"[OK] lm 已导出到: {lm_out_dir}")
#     print(f"[OK] 导出目录文件列表: {sorted(os.listdir(lm_out_dir))}")


# def main():
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     # 原始 T2Struc 最终模型目录（包含完整权重）
#     t2struc_final_model_dir = "/t9k/mnt/CM/weights/T2struc-1.2B"

#     # PINAL authors 官方训练并发布的 T5-large 子模块/权重目录（用于 tokenizer 回退复制）
#     pinal_official_t5_large_ckpt_dir = "/t9k/mnt/CM/text2amp/weights/pinal-official-t5-large/"

#     # 导出目录（建议不要和源目录同一路径，避免覆盖/混淆）
#     lm_out_dir = "/t9k/mnt/CM/text2amp/weights/pinal-official-t5-large/"

#     export_lm_only(
#         t2struc_final_model_dir=t2struc_final_model_dir,
#         pinal_official_t5_large_ckpt_dir=pinal_official_t5_large_ckpt_dir,
#         lm_out_dir=lm_out_dir,
#         device=device,
#         dtype=torch.float16,
#     )


# if __name__ == "__main__":
#     main()


In [1]:
import torch
from transformers import AutoTokenizer
from models.T5Encoder.t5_encoder_pinal import T5Encoder

def load_lm_only(
    lm_dir: str,
    device: torch.device,
    dtype: torch.dtype = torch.float16,
):
    tokenizer = AutoTokenizer.from_pretrained(lm_dir, use_fast=True)
    model = T5Encoder(lm_dir=lm_dir, device=device, dtype=dtype)
    return model, tokenizer


/t9k/mnt/.conda/envs/ref-progen3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lm_dir = "/t9k/mnt/CM/text2amp/weights/pinal-official-t5-large"

model, tokenizer = load_lm_only(lm_dir, DEVICE, dtype=torch.float16)

batch = tokenizer(
    ["hello world", "this is a test"],
    padding=True,
    return_tensors="pt",
)
batch = {k: v.to(DEVICE) for k, v in batch.items()}

# 1) 最常用：只拿最后一层 hidden states
with torch.no_grad():
    last_hidden = model(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        return_dict=False,   # 直接返回 Tensor
    )

print(last_hidden.shape)  # (B, L, H) -> (2, seq_len, 1024)

print(model)


torch.Size([2, 6, 1024])
T5Encoder(
  (lm): T5EncoderModel(
    (shared): Embedding(32128, 1024)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=1024, out_features=4096, bias=False)
                (wo)